In [1]:
#Import libraries 
import keras
import numpy as np
import pandas as pd
from keras.applications import resnet50
from keras import models
from keras import layers
from keras import optimizers
import cv2
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import os
from matplotlib import pyplot as plt
import numpy as np
from keras.callbacks import ModelCheckpoint
from PIL import Image

Using TensorFlow backend.


In [2]:
resnet_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
for layer in resnet_model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7ffb1891b668> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7ffb19e3e9e8> True
<keras.layers.convolutional.Conv2D object at 0x7ffb19e3eb00> True
<keras.layers.normalization.BatchNormalization object at 0x7ffb0887f080> True
<keras.layers.core.Activation object at 0x7ffaf9dde9e8> True
<keras.layers.convolutional.ZeroPadding2D object at 0x7ffaf9dfbbe0> True
<keras.layers.pooling.MaxPooling2D object at 0x7ffaf9ddec18> True
<keras.layers.convolutional.Conv2D object at 0x7ffaf9e83f60> True
<keras.layers.normalization.BatchNormalization object at 0x7ffb094269b0> True
<keras.layers.core.Activation object at 0x7ffb09426f98> True
<keras.layers.convolutional.Conv2D object at 0x7ffb09426d68> True
<keras.layers.normalization.BatchNormalization object at 0x7ffb2a856f60> True
<keras.layers.core.Activation object at 0x7ffb2a871898> True
<keras.layers.convolutional.Conv2D object at 0x7ffb2a8d3a90> True
<keras.layers.convolutional.Conv2D obj

In [4]:
# Adding classifier on top of Convolution model
# create model 
model = models.Sequential()

In [5]:
# Add the resnet50 convolutional model
model.add(resnet_model)

# Adding new Layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


In [6]:
# summary of model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2, 2, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              8389632   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10250     
Total params: 31,987,594
Trainable params: 31,934,474
Non-trainable params: 53,120
_________________________________________________________________


In [7]:
# Fitting the CNN to the images(Image Augmentation, Image Preprocessing)
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( 
        rescale=1./255,
        shear_range=0.2,             
        zoom_range=0.2,              
        horizontal_flip=True)

In [8]:
#Normalizing the test set
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
path = r'/Users/shiqinchoo/Desktop/satellite/rgb'
os.chdir(path)

In [10]:
train_path = r'/Users/shiqinchoo/Desktop/satellite/rgb/train'
training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(64,64),
    batch_size=32,
    class_mode = 'categorical'
)

Found 21600 images belonging to 10 classes.


In [11]:
test_path = r'/Users/shiqinchoo/Desktop/satellite/rgb/test'
test_set = train_datagen.flow_from_directory(
    test_path,
    target_size = (64,64),
    batch_size = 32,
    shuffle = False,
    class_mode = 'categorical'
)

Found 5400 images belonging to 10 classes.


In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [13]:
import tensorflow as tf
checkpoint_path = "/Users/shiqinchoo/Desktop/satellite/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

In [14]:
history = model.fit_generator(
    training_set, 
    steps_per_epoch = 100, 
    epochs = 50, verbose = 1, 
    validation_data = test_set, 
    validation_steps=500)

Epoch 1/50
  9/100 [=>............................] - ETA: 3:46 - loss: 2.6346 - acc: 0.3264

KeyboardInterrupt: 

In [19]:
model.save('sat_rgb.h5')

In [27]:
model = keras.models.load_model('/Users/shiqinchoo/Desktop/satellite/rgb/sat_rgb.h5')

In [ ]:
Y_pred = model.predict_generator(test_set, test_set.samples)
y_pred = np.argmax(Y_pred, axis = 1)

In [28]:
y_pred.shape

(172552,)

In [21]:
Y_pred.shape

(172552, 10)

In [22]:
test_set.classes.shape

(5400,)

In [24]:
print(test_set.samples)

5400


In [26]:
print('Confusion Matrix')

cm = confusion_matrix(test_set.class_indices, y_pred)

print (cm)

Confusion Matrix


ValueError: Found input variables with inconsistent numbers of samples: [10, 172552]

In [19]:
print('Confusion Matrix')

cm = confusion_matrix(test_set.classes, y_pred)

print (cm)

Confusion Matrix


ValueError: Found input variables with inconsistent numbers of samples: [5400, 172552]